# Opening images in Keras
Todo:
 - Change from Pillow to rasterio to read image
 - 

Sources:
    - https://medium.com/@ensembledme/writing-custom-keras-generators-fe815d992c5a
    - https://stackoverflow.com/questions/46493419/use-a-generator-for-keras-model-fit-generator
    - https://stackoverflow.com/questions/50195967/how-can-i-create-a-4d-numpy-array-from-images
    - https://www.google.com/search?client=firefox-b-ab&q=keras+make+image+generator
 

In [46]:
from PIL import Image

import keras
from PIL import Image


In [ ]:
#!pip install git+git://github.com/JonHidlePedersen/keras-preprocessing.git

In [47]:
import rasterio
import numpy as np

with rasterio.open('../Sorted_tiles/testimage.tif', 'r') as ds:
    arr = ds.read()  # read all raster values
    #arr = np.moveaxis(arr, 0, -1) # Moves the channel index to the back.

print(arr.shape) 

import imageio
im = imageio.imread('../Sorted_tiles/testimage.tif')
print(im.shape) 

RasterioIOError: ../Sorted_tiles/testimage.tif: No such file or directory

In [48]:
import glob
import os
list_images = glob.glob('../Sorted_tiles/*.tif')

batch_paths = np.random.choice(a = list_images, 
                                         size = 2)
print(batch_paths)
print(os.path.basename(batch_paths[0]))

['../Sorted_tiles/20180705-biri_13_SWIR_384me_SN3126_raw_rad_bsq_float32_geo_007_024.tif'
 '../Sorted_tiles/20180705-biri_13_SWIR_384me_SN3126_raw_rad_bsq_float32_geo_011_029.tif']
20180705-biri_13_SWIR_384me_SN3126_raw_rad_bsq_float32_geo_007_024.tif


In [48]:
k = np.random.randint(low=1, high=6,  size=30)
print(k)

[4 2 4 2 5 5 2 5 4 1 3 5 1 4 5 3 4 1 1 3 3 5 4 3 5 3 1 3 4 5]


In [79]:
def flip_vertical(image):
    return np.flipud(image)

def flip_horizontal(image):
    return np.fliplr(image)

def rotate_90(image):
    return np.rot90(image, k=1)

def rotate_180(image):
    return np.rot90(image, k=2)

def rotate_270(image):
    return np.rot90(image, k=3)

def random_image_flip_rotation(image):
    # This function rotates or flips the image randomly using one of five functions. 
    # This function can also do nothing to the image. 1/6 chanche.
    # Returns a image

    randint = np.random.randint(low=1, high=6)
    modified_image = 0
    
    if randint == 1:
        modified_image = flip_vertical(image)
    elif randint == 2:
        modified_image = flip_horizontal(image)

    elif randint == 3:
        modified_image = rotate_90(image)

    elif randint == 4:
        modified_image = rotate_180(image)

    elif randint == 5:
        modified_image = rotate_270(image)

    else:
        modified_image = image
        
    return modified_image

A = np.array([[1, 2, 3], [4, 5, 6], [7,8,9]])
print(A)
print(flip_vertical(A))
print(flip_horizontal(A))
print(rotate_90(A))
print(rotate_180(A))
print(rotate_270(A))
print(random_image_flip_rotation(A))

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[7 8 9]
 [4 5 6]
 [1 2 3]]
[[3 2 1]
 [6 5 4]
 [9 8 7]]
[[3 6 9]
 [2 5 8]
 [1 4 7]]
[[9 8 7]
 [6 5 4]
 [3 2 1]]
[[7 4 1]
 [8 5 2]
 [9 6 3]]
[[7 4 1]
 [8 5 2]
 [9 6 3]]


In [74]:
def get_input(path):
    
    img = imageio.imread(path)
    
    return(img)


def image_generator(images_path, label_path, batch_size = 3):
    # Check that all images are the same size. Dimentions
    
    while True:
        # Select files (paths/indices) for the batch
        batch_paths = np.random.choice(a = images_path, 
                                     size = batch_size)
        batch_input = []
        batch_y = 0
        batch_x = 0

        # Read in each input, perform preprocessing and get labels

        for input_path in batch_paths:
            # Reads image from file
            input_img = get_input(input_path)

            # Randomly flips/rotates/does nothing to the image
            input_img = random_image_flip_rotation(input_img)


            batch_input += [ input_img ]

            # Set a label by using the tile name
            label = int(labelfile[labelfile.Tile_name == os.path.basename(input_path)]['LIVSM1'])

            batch_y.append(int(label)) # Needs to be int not string

        # Return a tuple of (input,output) to feed the network
        batch_x = np.stack(batch_input, axis=0)

        yield(batch_x, batch_y)

In [49]:
import pandas as pd
labelfile = pd.read_csv('../Sorted_tiles/labeled_tiles.csv')
labelfile.head()

,ID,Tile_name,LIVSM1,LIVSM2
0,0,20180705-biri_13_SWIR_384me_SN3126_raw_rad_bsq...,0,0
1,1,20180705-biri_13_SWIR_384me_SN3126_raw_rad_bsq...,0,0
2,2,20180705-biri_13_SWIR_384me_SN3126_raw_rad_bsq...,0,0
3,3,20180705-biri_13_SWIR_384me_SN3126_raw_rad_bsq...,0,0
4,4,20180705-biri_13_SWIR_384me_SN3126_raw_rad_bsq...,0,0


In [ ]:
MultiLabelBinarizer is used to make a multilabel binary label. This labelvector is used by the model.
from sklearn.preprocessing import MultiLabelBinarizer

mergedlabels = [list(pair) for pair in zip(labelfile['LIVSM1'].values, labelfile['LIVSM2'].values)]


mlb = MultiLabelBinarizer(classes=[1,2,3,4,5,6,7,8,9,10,11,12,99])
binarized_labels = mlb.fit_transform(mergedlabels)

labeled_tiles["Multi_label_binarized"] = binarized_labels

In [50]:
# kk = labelfile.loc[labelfile['ID'] == 0]['LIVSM1']
kk = labelfile[labelfile.ID == 0]['LIVSM1']
print(int(kk))
print(kk + 1)

0
0    1
Name: LIVSM1, dtype: int64


In [55]:
list_images = glob.glob('../Sorted_tiles/*.tif')


def get_input(path):
    
    img = imageio.imread(path)
    
    return(img)


def image_generator(list_images, label_file, batch_size = 3):
    # list_images: list of where the images are placed.
    # label_file: csv file with labels and filenames.
    
    # Check that all images are the same size. Dimentions
    while True:
        batch_input = []
        batch_y = []
        batch_x = 0
        
        
        # Select files (paths/indices) for the batch
        batch_paths = np.random.choice(a = list_images, 
                                     size = batch_size)

        # Read in each input, perform preprocessing and get labels

        for input_path in batch_paths:

          input = get_input(input_path)
          batch_input += [ input ]

          # Set a label by using the tile name
          label = int(labelfile[labelfile.Tile_name == os.path.basename(input_path)]['LIVSM1'])

          batch_y.append(int(label)) # Needs to be int not string

        # Return a tuple of (input,output) to feed the network
        batch_x = np.stack(batch_input, axis=0)

        yield(batch_x, batch_y)

In [58]:
test_generated_images = image_generator(list_images, labelfile)
generert = next(test_generated_images)
print(generert)
#print(next(test_generated_images))
print(np.shape(generert[0]))
print(np.shape(generert[1]))
print(generert[1])

(array([[[[2.00054562e-03, 3.24347918e-03, 4.13193181e-03, ...,
          2.06467594e-05, 1.99686227e-04, 1.82959600e-04],
         [2.92097102e-03, 4.05124808e-03, 5.13031334e-03, ...,
          7.01284662e-05, 0.00000000e+00, 1.39926065e-04],
         [3.68173537e-03, 5.75274229e-03, 7.14319758e-03, ...,
          2.61318510e-05, 0.00000000e+00, 0.00000000e+00],
         ...,
         [1.04343481e-02, 1.53941521e-02, 1.86565742e-02, ...,
          0.00000000e+00, 4.68106591e-05, 1.25125385e-04],
         [5.93545288e-03, 8.39048158e-03, 1.04681253e-02, ...,
          1.41328026e-04, 2.42041453e-04, 1.25110018e-04],
         [9.00670793e-03, 1.30653074e-02, 1.71106569e-02, ...,
          1.35824390e-04, 6.50979928e-05, 2.46987125e-04]],

        [[2.89609353e-03, 4.51174192e-03, 5.57992468e-03, ...,
          0.00000000e+00, 0.00000000e+00, 1.40091972e-04],
         [3.62550747e-03, 5.05697355e-03, 6.55726809e-03, ...,
          1.51159866e-05, 2.55198192e-05, 7.45862635e-05],
       

In [59]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(50, 50, 288)))
model.add(Activation('relu'))

model.add(GlobalAveragePooling2D())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 48, 48, 32)        82976     
_________________________________________________________________
activation_5 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
global_average_pooling2d_3 ( (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               8448      
_________________________________________________________________
activation_6 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
Total para

In [62]:
model.fit_generator(
        image_generator(list_images, labelfile),
        steps_per_epoch=5,
        epochs=10)

Epoch 1/10
5/5 [==============================] - 2s 386ms/step - loss: 0.7181 - acc: 0.8000
Epoch 2/10


ValueError: all input arrays must have the same shape

In [14]:
nn = model.predict_generator(
        image_generator(list_images),
        steps=10)

array([[0.50386584],
       [0.50610065],
       [0.5037968 ],
       [0.5038453 ],
       [0.5058131 ],
       [0.5042402 ],
       [0.5049603 ],
       [0.5042402 ],
       [0.5037968 ],
       [0.5030699 ],
       [0.5027083 ],
       [0.5042462 ],
       [0.5037197 ],
       [0.5038573 ],
       [0.504117  ],
       [0.5065854 ],
       [0.5042402 ],
       [0.50286454],
       [0.503915  ],
       [0.5030699 ],
       [0.50330627],
       [0.5042402 ],
       [0.5036756 ],
       [0.503312  ],
       [0.5045645 ],
       [0.50428814],
       [0.5038453 ],
       [0.5058131 ],
       [0.5035124 ],
       [0.50544775]], dtype=float32)

In [19]:
#img = Image.open('../Sorted_tiles/kkkk.tiff')

from osgeo import gdal
import numpy as np
ds = gdal.Open('../Sorted_tiles/testimage.tif')
myarray = np.array(ds.GetRasterBand(1).ReadAsArray())

()


In [ ]:
# Finne ut korleis eg får lest det slik at matrisa blir 4 dimensjonal.
# Check that all images are the same size. Dimentions